In [15]:
import os
import glob
import pandas as pd
import numpy as np
from scipy.stats import wilcoxon

# Getting subject data folders from server
server = r'/Volumes/greenhouse/typingtask_data/subject_data'
os.chdir(server)

# sub_folders_turbo = list(pd.read_csv(
#     r'/Users/rubi/Desktop/Github/typingexp/typing_task_analysis/manuscript_IDs.csv'
# )['with_turbo'])

all_bytrial = pd.DataFrame()
all_bybigram = pd.DataFrame()
ID_list = []

sub_folders = glob.glob('s*')

# Loop through subjects
for sub in sub_folders:
    sub_folder = rf'/Volumes/greenhouse/typingtask_data/subject_data/{sub}/psychopy_data/edited'
    os.chdir(sub_folder)
    sID = sub.split('_', 1)[0]
    bytrial_files = glob.glob('*_bytrial.csv')
    bytrial = pd.read_csv(bytrial_files[0])
    bybigram_files = glob.glob('*_bybigram.csv')
    # if not bytrial_files:
        # continue
    bybigram = pd.read_csv(bybigram_files[0])
    bytrial['Subject'] = sID
    all_bytrial = pd.concat([all_bytrial, bytrial])
    all_bybigram = pd.concat([all_bybigram, bybigram])
    ID_list.append(sID)

  # bybigram = pd.read_csv(glob.glob('*_bybigram.csv')[0])
  #   all_bytrial = pd.concat([all_bytrial, bytrial])
  #   all_bybigram = pd.concat([all_bybigram, bybigram])
  #   ID_list.append(sID)

# # List of subjects without turbo typing data
# server_noturbo = r'/Volumes/greenhouse/typingtask_data/subject_data/not_used/no_turbotyping/'
# os.chdir(server_noturbo)
# sub_folders_noturbo = list(pd.read_csv(
#     r'/Users/rubi/Desktop/Github/typingexp/typing_task_analysis/manuscript_IDs.csv'
# )['without_turbo'].dropna())

# # Loop through non-turbo subjects
# for sub in sub_folders_noturbo:
#     sub_folder = server_noturbo + rf'{sub}/psychopy_data/edited'
#     os.chdir(sub_folder)
#     sID = sub.split('_', 1)[0]
#     bytrial_files = glob.glob('*_bytrial.csv')
#     if not bytrial_files:
#         continue
#     bytrial = pd.read_csv(bytrial_files[0])
#     bytrial['Subject'] = sID
#     all_bytrial = pd.concat([all_bytrial, bytrial], ignore_index=True)
#     ID_list.append(sID)

print(f"✅ Loaded {len(ID_list)} subjects, total rows: {len(all_bytrial)}")


✅ Loaded 34 subjects, total rows: 8160


In [ ]:
# iki_metrics = [stat.mean, stat.stdev, variation]

# DF = dataframe[dataframe.bigram.isin(group)]
# metrics = DF.groupby('sID').IKI.agg(metric_list)

In [2]:
target_word = "zibja"
rt_cols = ['key_resp.rt.1', 'key_resp.rt.2', 'key_resp.rt.3', 'key_resp.rt.4', 'key_resp.rt.5']
ALPHA = 0.05

# Filter for correct trials and target word
df_core = all_bytrial[(all_bytrial['resp_string'] == all_bytrial['string']) &
                      (all_bytrial['string'] == target_word)].copy()

ikis = []
for _, row in df_core.iterrows():
    rts = pd.to_numeric(row[rt_cols], errors='coerce').values
    if not np.all(np.isfinite(rts)):
        continue
    diffs = np.diff(rts)
    ikis.append(diffs)

MAT = np.vstack(ikis)

rows = []
boundaries = []

for i in range(MAT.shape[1] - 1):
    x = MAT[:, i]
    y = MAT[:, i + 1]
    stat, p = wilcoxon(x, y, zero_method="pratt")
    mean_x = float(np.mean(x))
    mean_y = float(np.mean(y))
    direction = "ns"
    if p < ALPHA:
        direction = "increase" if mean_y > mean_x else "decrease"
        if direction == "increase":
            boundaries.append(i + 3)
            
    rows.append({
        "Compare": f"IKI{i+1} (press {i+1}->{i+2}) vs IKI{i+2} (press {i+2}->{i+3})",
        "mean_n": round(mean_x, 3),
        "mean_next": round(mean_y, 3),
        "p_value": f"{p:.3e}",
        "direction": direction,
        "n_trials": MAT.shape[0],
    })

results_df = pd.DataFrame(rows, columns=["Compare", "mean_n", "mean_next", "p_value", "direction", "n_trials"])
print(f"\nWilcoxon results on successive IKIs for '{target_word}' (all trials):")
print(results_df.to_string(index=False))

chunks = []
start = 1
for b in sorted(set(boundaries)):
    chunks.append([start, b - 1])
    start = b
chunks.append([start, 5])

valid_chunks = [c for c in chunks if (c[1] - c[0] + 1) >= 2]
fmt = lambda a, b: f"{a}–{b}" if a != b else f"{a}"

print("\nInferred chunk ranges (tentative):", ", ".join(fmt(a, b) for a, b in chunks))
print("Valid chunks (length ≥ 2):", "none" if not valid_chunks else ", ".join(fmt(a, b) for a, b in valid_chunks))



Wilcoxon results on successive IKIs for 'zibja' (all trials):
                               Compare  mean_n  mean_next   p_value direction  n_trials
IKI1 (press 1->2) vs IKI2 (press 2->3)   0.191      0.289 1.261e-26  increase       336
IKI2 (press 2->3) vs IKI3 (press 3->4)   0.289      0.240 7.895e-07  decrease       336
IKI3 (press 3->4) vs IKI4 (press 4->5)   0.240      0.132 1.043e-47  decrease       336

Inferred chunk ranges (tentative): 1–2, 3–5
Valid chunks (length ≥ 2): 1–2, 3–5
